# Jupyter Notebook to Convert the Spreadsheet CSV in Mapping dictionaries

In [1]:
import numpy as np
import pandas as pd
import os
import sys

In [3]:
os.getcwd()

'/Users/peterkinget/Library/Mobile Documents/com~apple~CloudDocs/BOX/Work/MOSBIUS/MOSbiusTools/MOSbiusV2Tools/commandline/jupyter_notebooks_basis_for_scripts'

In [13]:
# Change the working directory to the script's directory

working_dir = "/Users/peterkinget/iCloudDrive/Work/MOSBIUS/MOSbiusTools/MOSbiusV2Tools/commandline/chip_config_data/csv_sources"
os.chdir(working_dir)

# Define the file path
filename = "MOSbiusV2_pin_map - Pin_List_20250515_cleaned.csv"

# Check if the script is being run from the correct directory
if not os.path.exists(filename):
    print(f"Error: The script must be run from the directory containing {filename}.")
    sys.exit(1) 

In [15]:
# Read the CSV file
df_orig = pd.read_csv(filename, skiprows=0)  # Skip the first 10 rows of metadata

In [16]:
columns = df_orig.columns.tolist()   # Print the column names
columns

['pin_number', 'sw_matrix_pin_number', 'pin_name']

In [17]:
df_orig

,pin_number,sw_matrix_pin_number,pin_name
0,1.0,92.0,VSS
1,2.0,NaN,EN
2,3.0,NaN,CLK
3,4.0,NaN,DATA_SBUS6
4,5.0,NaN,SBUS5
...,...,...,...
103,97.0,87.0,DCC4_P_D_L_CC
104,98.0,88.0,DCC4_P_D_L_CS
105,99.0,89.0,DCC4_P_D_R_CC
106,100.0,90.0,DCC4_P_D_R_CS


In [20]:
df = df_orig.dropna(how='all')
df

,pin_number,sw_matrix_pin_number,pin_name
0,1.0,92.0,VSS
1,2.0,NaN,EN
2,3.0,NaN,CLK
3,4.0,NaN,DATA_SBUS6
4,5.0,NaN,SBUS5
...,...,...,...
103,97.0,87.0,DCC4_P_D_L_CC
104,98.0,88.0,DCC4_P_D_L_CS
105,99.0,89.0,DCC4_P_D_R_CC
106,100.0,90.0,DCC4_P_D_R_CS


In [21]:
# save_columns = ['Package Pin no.', 'Switch Matrix Pin No. ', 'Pin Name', ]
# # Filter the DataFrame to include only the specified columns
# df = df_orig[save_columns]
# # Rename the columns
# df.rename(columns={
#     'Package Pin no.': 'pin_number',
#     'Switch Matrix Pin No. ': 'sw_matrix_pin_number',
#     'Pin Name': 'pin_name'
# }, inplace=True)    
# # Remove the empty rows
# df = df.dropna(how='all')

In [22]:
print(df)
print(df.columns)

     pin_number  sw_matrix_pin_number       pin_name
0           1.0                  92.0            VSS
1           2.0                   NaN             EN
2           3.0                   NaN            CLK
3           4.0                   NaN     DATA_SBUS6
4           5.0                   NaN          SBUS5
..          ...                   ...            ...
103        97.0                  87.0  DCC4_P_D_L_CC
104        98.0                  88.0  DCC4_P_D_L_CS
105        99.0                  89.0  DCC4_P_D_R_CC
106       100.0                  90.0  DCC4_P_D_R_CS
107         NaN                  92.0            NaN

[108 rows x 3 columns]
Index(['pin_number', 'sw_matrix_pin_number', 'pin_name'], dtype='object')


In [26]:
# Make a dictionary from the DataFrame that maps Pin Name to Pin No.
pin_map = {}
for index, row in df.iterrows():
    pin_name = row['pin_name']
    pin_number = row['pin_number']
    if pd.notna(pin_name) and pd.notna(pin_number):
        pin_map[pin_name] = int(pin_number)
pin_name_to_number = pin_map

# Make a dictionary from the Dataframe that maps Pin numbers to pin names
pin_number_to_name = {}
for index, row in df.iterrows():
    pin_number = row['pin_number']
    pin_name = row['pin_name']
    if pd.notna(pin_number) and pd.notna(pin_name):
        pin_number_to_name[int(pin_number)] = pin_name

# Make a dictionary from the Dataframe that maps the pin name to the switch matrix pin number
sw_matrix_pin_map = {}
for index, row in df.iterrows():
    pin_name = row['pin_name']
    sw_matrix_pin_number = row['sw_matrix_pin_number']
    if pd.notna(pin_name) and pd.notna(sw_matrix_pin_number):
        sw_matrix_pin_map[pin_name] = int(sw_matrix_pin_number)

pin_name_to_sw_matrix_pin_number = sw_matrix_pin_map

In [27]:
sw_matrix_pin_map

{'VSS': 92,
 'OTA_P_INP': 1,
 'OTA_P_INN': 2,
 'OTA_P_OUT': 3,
 'VDD': 91,
 'OTA_N_INP': 4,
 'OTA_N_INN': 5,
 'OTA_N_OUT': 6,
 'DCC3_N_G_L_CC': 7,
 'DCC3_N_G_L_CS': 8,
 'DCC3_N_G_R_CC': 9,
 'DCC3_N_G_R_CS': 10,
 'DCC3_N_D_L_CC': 11,
 'DCC3_N_D_L_CS': 12,
 'DCC3_N_D_R_CC': 13,
 'DCC3_N_D_R_CS': 14,
 'DCC4_N_G_L_CC': 15,
 'DCC4_N_G_L_CS': 16,
 'DCC4_N_G_R_CC': 17,
 'DCC4_N_G_R_CS': 18,
 'DCC4_N_D_L_CC': 19,
 'DCC4_N_D_L_CS': 20,
 'DCC4_N_D_R_CC': 21,
 'DCC4_N_D_R_CS': 22,
 'DCC2_N_G_L_CC': 23,
 'DCC2_N_G_L_CS': 24,
 'DCC2_N_G_R_CC': 25,
 'DCC2_N_G_R_CS': 26,
 'DCC2_N_D_L_CC': 27,
 'DCC2_N_D_L_CS': 28,
 'DCC2_N_D_R_CC': 29,
 'DCC2_N_D_R_CS': 30,
 'DCC1_N_G_L_CC': 31,
 'DCC1_N_G_L_CS': 32,
 'DCC1_N_G_R_CC': 33,
 'DCC1_N_G_R_CS': 34,
 'DCC1_N_D_L_CC': 35,
 'DCC1_N_D_L_CS': 36,
 'DCC1_N_D_R_CC': 37,
 'DCC1_N_D_R_CS': 38,
 'DINV2_INP_L': 39,
 'DINV2_INN_L': 40,
 'DINV2_OUT_L': 41,
 'DINV2_INP_R': 42,
 'DINV2_INN_R': 43,
 'DINV2_OUT_R': 44,
 'DINV1_INP_L': 45,
 'DINV1_INN_L': 46,
 'DINV1_OUT_L

In [28]:
# Save the dictionaries to JSON files
import json 
with open('pin_name_to_number.json', 'w') as f:
    json.dump(pin_name_to_number, f, indent=4)
with open('pin_number_to_name.json', 'w') as f:
    json.dump(pin_number_to_name, f, indent=4)  
with open('pin_name_to_sw_matrix_pin_number.json', 'w') as f:   
    json.dump(pin_name_to_sw_matrix_pin_number, f, indent=4)
# Save the DataFrame to a new CSV file
df.to_csv('MOSbiusV2_pin_map - Pin List_cleaned.csv', index=False)

In [ ]:
pin_name_to_number 

In [ ]:
pin_number_to_name

In [ ]:
pin_name_to_sw_matrix_pin_number

## Size Registers

In [73]:
import pandas as pd

def group_register_bits(csv_file_path):
    """
    Reads a CSV file and groups register bits into sets of 5 bits for each device.

    Args:
        csv_file_path (str): Path to the CSV file.

    Returns:
        dict: A dictionary where keys are device names and values are dictionaries
              mapping bit labels (1, 2, 4, 8, 16) to register numbers.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Initialize a dictionary to store grouped registers
    grouped_registers = {}

    # Iterate through each row in the DataFrame
    for _, row in df.iterrows():
        reg_name = row['Reg Names']
        reg_number = row['Reg Numbers']

        # Extract the base device name (e.g., "DCC1_P_L" from "DCC1_P_L_1")
        base_name = "_".join(reg_name.split("_")[:-1])

        # Extract the bit label (e.g., "1" from "DCC1_P_L_1")
        bit_label = int(reg_name.split("_")[-1])

        # Add the register to the corresponding device group
        if base_name not in grouped_registers:
            grouped_registers[base_name] = {}
        grouped_registers[base_name][bit_label] = reg_number

    # Ensure each group is sorted by the bit label
    for base_name in grouped_registers:
        grouped_registers[base_name] = dict(sorted(grouped_registers[base_name].items()))

    return grouped_registers

# Example usage
csv_file_path = "MOSbiusV2_register_map - analog_sizing_20250515.csv"
grouped_registers = group_register_bits(csv_file_path)

# Print the grouped registers
for device, bits in grouped_registers.items():
    print(f"{device}: {bits}")
    

OTA_P: {1: 1889, 2: 1890, 4: 1891, 8: 1892, 16: 1893}
DCC1_P_L: {1: 1894, 2: 1895, 4: 1896, 8: 1897, 16: 1898}
DCC1_P_R: {1: 1899, 2: 1900, 4: 1901, 8: 1902, 16: 1903}
DCC2_P_L: {1: 1904, 2: 1905, 4: 1906, 8: 1907, 16: 1908}
DCC2_P_R: {1: 1909, 2: 1910, 4: 1911, 8: 1912, 16: 1913}
DCC3_P_L: {1: 1914, 2: 1915, 4: 1916, 8: 1917, 16: 1918}
DCC3_P_R: {1: 1919, 2: 1920, 4: 1921, 8: 1922, 16: 1923}
DCC4_P_L: {1: 1924, 2: 1925, 4: 1926, 8: 1927, 16: 1928}
DCC4_P_R: {1: 1929, 2: 1930, 4: 1931, 8: 1932, 16: 1933}
CC_N: {1: 1934, 2: 1935, 4: 1936, 8: 1937, 16: 1938}
CC_P: {1: 1939, 2: 1940, 4: 1941, 8: 1942, 16: 1943}
DINV1_L: {1: 1944, 2: 1945, 4: 1946, 8: 1947, 16: 1948}
DINV1_R: {1: 1949, 2: 1950, 4: 1951, 8: 1952, 16: 1953}
DINV2_L: {1: 1954, 2: 1955, 4: 1956, 8: 1957, 16: 1958}
DINV2_R: {1: 1959, 2: 1960, 4: 1961, 8: 1962, 16: 1963}
DCC1_N_L: {1: 1964, 2: 1965, 4: 1966, 8: 1967, 16: 1968}
DCC1_N_R: {1: 1969, 2: 1970, 4: 1971, 8: 1972, 16: 1973}
DCC2_N_L: {1: 1974, 2: 1975, 4: 1976, 8: 1977,

In [74]:
# Save the dictionaries to JSON files

with open('device_name_to_sizing_registers.json', 'w') as f:
    json.dump(grouped_registers, f, indent=4)


## Generating Switch Matrix Register Map

In [36]:
import csv
import json
import os

In [33]:
# Change the working directory to the script's directory

working_dir = "/Users/peterkinget/iCloudDrive/Work/MOSBIUS/MOSbiusTools/MOSbiusV2Tools/commandline/chip_config_data/csv_sources/"
os.chdir(working_dir)

# Define the file path
file_path = "MOSbiusV2_register_map - swmatrix_20250515.csv"

# Check if the script is being run from the correct directory
if not os.path.exists(file_path):
    print(f"Error: The script must be run from the directory containing {file_path}.")
    sys.exit(1) 

In [50]:
def parse_csv_to_switch_matrix_register_map_old(file_path):
    """
    Parses the CSV file into a nested dictionary for easy lookup by pin/internal_pin and bus type.
    
    Args:
        file_path (str): Path to the CSV file.
    
    Returns:
        dict: A nested dictionary where keys are pin/internal_pin names, and values are dictionaries
              mapping bus types (SBUS or RBUS) and columns (a, b, or RBUS index) to register numbers.
    """
    register_map = {}

    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)
        headers = next(csv_reader)  # Read the header row
        headers = next(csv_reader)  # Read the header row
        bus_headers = headers[2:-1]  # Extract bus headers (SBUS and RBUS columns)

        for row in csv_reader:
            if not any(row):  # Skip empty rows
                continue

            pin_type = row[0].strip()  # e.g., "swmatrix_pin" or "swmatrix_int_pin"
            pin_name = row[1].strip()  # e.g., "1", "A", etc.

            if pin_type and pin_name:
                pin_key = f"{pin_type}_{pin_name}"
                pin_key = pin_name
                register_map[pin_key] = {}

                for i, bus_header in enumerate(bus_headers):
                    if bus_header:  # Skip empty headers
                        # bus_type, column = bus_header[:-1], bus_header[-1]  # e.g., "SBUS1a" -> "SBUS1", "a"
                        register_value = row[i + 2].strip()  # Offset by 2 to match column index
                        if register_value:  # Only add non-empty register values
                            register_map[pin_key][bus_header] = register_value
    return register_map


In [56]:
def parse_csv_to_switch_matrix_register_map(file_path):
    """
    Parses the CSV file into a nested dictionary for easy lookup by pin/internal_pin and bus type.
    
    Args:
        file_path (str): Path to the CSV file.
    
    Returns:
        dict: A nested dictionary where keys are pin/internal_pin names, and values are dictionaries
              mapping bus types (SBUS or RBUS) and columns (a, b, or RBUS index) to register numbers.
    """
    register_map = {}

    with open(file_path, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Read the first row with bus types (SBUS, RBUS)
        bus_types_row = next(csv_reader)
        
        # Read the second row with column identifiers (1a, 1b, 1, 2, etc.)
        column_ids_row = next(csv_reader)
        
        # Create combined header names (SBUS1a, RBUS1, etc.)
        combined_headers = []
        current_bus_type = ""
        
        for i in range(2, len(bus_types_row) - 1):  # Skip first 2 columns and last column
            bus_type = bus_types_row[i].strip()
            col_id = column_ids_row[i].strip()
            
            # If a non-empty bus_type is found, update the current_bus_type
            if bus_type:
                current_bus_type = bus_type
                
            # If we have both a bus type and column id, create a combined header
            if current_bus_type and col_id:
                combined_headers.append(f"{current_bus_type}{col_id}")
            else:
                combined_headers.append("")  # Keep empty placeholder for alignment
        
        print(f"Generated headers: {[h for h in combined_headers if h]}")
        
        # Process data rows
        for row in csv_reader:
            if not any(row):  # Skip empty rows
                continue

            pin_type = row[0].strip()  # e.g., "swmatrix_pin" or "swmatrix_int_pin"
            pin_name = row[1].strip()  # e.g., "1", "A", etc.
            
            # Get the pin display name from the last column if available
            pin_display_name = row[-1].strip() if len(row) > 2 else ""

            if pin_type and pin_name:
                # Use pin_name as the key (A, B, 1, 2, etc.)
                register_map[pin_name] = {}

                for i, header in enumerate(combined_headers):
                    if header and i + 2 < len(row):  # Skip empty headers and ensure index is valid
                        register_value = row[i + 2].strip()  # Offset by 2 to match column index
                        if register_value and register_value.lower() != 'x':  # Only add non-empty and non-x values
                            register_map[pin_name][header] = register_value
                
                # Add display name if available
                if pin_display_name:
                    register_map[pin_name]["display_name"] = pin_display_name
                
    return register_map

In [70]:
switch_matrix_register_map = parse_csv_to_switch_matrix_register_map(file_path)

# print(switch_matrix_register_map)
# print(switch_matrix_register_map.keys())
# print(switch_matrix_register_map["A"])

# Query example
pin = "A"
bus = "SBUS1a"
print(f"Register for {pin} on {bus} is {switch_matrix_register_map[pin][bus]}")
pin = "22"
bus = "SBUS5b"
print(f"Register for {pin} on {bus} is {switch_matrix_register_map[pin][bus]}")

Generated headers: ['SBUS1a', 'SBUS1b', 'SBUS2a', 'SBUS2b', 'SBUS3a', 'SBUS3b', 'SBUS4a', 'SBUS4b', 'SBUS5a', 'SBUS5b', 'SBUS6a', 'SBUS6b', 'RBUS1', 'RBUS2', 'RBUS3', 'RBUS4', 'RBUS5', 'RBUS6', 'RBUS7', 'RBUS8']
Register for A on SBUS1a is 47
Register for 22 on SBUS5b is 236


In [71]:
# Save the dictionaries to JSON files

with open('switch_matrix_register_map.json', 'w') as f:
    json.dump(switch_matrix_register_map, f, indent=4)


In [ ]:
%pwd